In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip',delimiter='\t')
df_test = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip',delimiter='\t')


In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()
X_train = df_train['Phrase'].values
Y_train = df_train['Sentiment'].values
X_test = df_test['Phrase'].values

In [ ]:
print(df_train['Sentiment'].value_counts())

In [ ]:
import string
import nltk
def clean_text(line):
    line = line.lower()
    no_punct = [words for words in line if words not in string.punctuation]
    line = ''.join(no_punct)
    line = line.split()
    line = [words for words in line if words not in nltk.corpus.stopwords.words('english')]
    line = ''.join(line)
    return line

In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def clean_text_stem(line):
    line = line.lower()
    no_punct = [words for words in line if words not in string.punctuation]
    line = ''.join(no_punct)
    line = line.split()
    line = [lemmatizer.lemmatize(words) for words in line if words not in nltk.corpus.stopwords.words('english')]
    line = ''.join(line)
    return line

In [ ]:
import tqdm
X_train_clean = [clean_text_stem(line) for line in tqdm.tqdm(X_train)]
X_test_clean = [clean_text_stem(line) for line in tqdm.tqdm(X_test)]

# **BAG OF WORDS**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train_clean)
X_train_cv = vectorizer.transform(X_train_clean)
X_test_cv = vectorizer.transform(X_test_clean)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_cv,X_val_cv,Y_train_cv,Y_val_cv = train_test_split(X_train_cv,Y_train,test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
nb_cv =  MultinomialNB().fit(X_train_cv, Y_train_cv)
clf = LogisticRegression(random_state=0,solver='lbfgs',class_weight='balanced', max_iter=10000).fit(X_train_cv, Y_train_cv)

In [ ]:
from sklearn.metrics import accuracy_score
Y_pred_cv = clf.predict(X_val_cv)
print(accuracy_score(Y_val_cv, Y_pred_cv))
Y_pred_cv = nb_cv.predict(X_val_cv)
print(accuracy_score(Y_val_cv, Y_pred_cv))

# **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(X_train_clean)
X_train_clean_tf = tfv.transform(X_train_clean)
X_test_clean_tf = tfv.transform(X_test_clean)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_tf,X_val_tf,Y_train_tf,Y_val_tf = train_test_split(X_train_clean_tf,Y_train,test_size=0.2)

In [ ]:
clf_tf = LogisticRegression(random_state=0,solver='lbfgs',class_weight='balanced', max_iter=10000).fit(X_train_tf, Y_train_tf)
nb_tf =  MultinomialNB().fit(X_train_tf, Y_train_tf)

In [ ]:
Y_pred_tf = clf_tf.predict(X_val_tf)
print(accuracy_score(Y_val_tf, Y_pred_tf))
Y_pred_tf = nb_tf.predict(X_val_tf)
print(accuracy_score(Y_val_tf, Y_pred_tf))

# **LSTM WORD2VEC**

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

t = Tokenizer()
t.fit_on_texts(X_train_clean)
X_train_token = t.texts_to_sequences(X_train_clean)
X_test_token = t.texts_to_sequences(X_test_clean)
sent_length = 50
X_train_token_pad = pad_sequences(X_train_token,padding='pre',maxlen=sent_length)
X_test_token_pad = pad_sequences(X_test_token,padding='pre',maxlen=sent_length)

In [ ]:
import keras
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense,SpatialDropout1D,Dropout
embedding_vector_features=100
voc_size = len(t.word_index)+1
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



In [ ]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
X_train_token_pad, X_val_token_pad, Y_train_token_pad, Y_val_token_pad = train_test_split(X_train_token_pad, Y_train, test_size=0.33, random_state=42)
Y_train_token_pad = np_utils.to_categorical(Y_train_token_pad)
Y_val_token_pad = np_utils.to_categorical(Y_val_token_pad)
model.fit(X_train_token_pad,Y_train_token_pad,validation_data=(X_val_token_pad,Y_val_token_pad),epochs=8,batch_size=64)
Y_pred_token_pre = model.predict_classes(X_test_token_pad)

# **LSTM pre-trained WORD2VEC**

In [ ]:
import tqdm
def get_vocab(data):
    vocab = {}
    for line in tqdm.tqdm(data):
        line = line.split()
        for word in line:
            try:
                vocab[word] += 1
            except:
                vocab[word] = 1
    return vocab

In [ ]:
vocab = get_vocab(X_train)

In [ ]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)

In [ ]:
from gensim.models import KeyedVectors
model_google = KeyedVectors.load_word2vec_format(path, binary = True);

In [ ]:
import operator 

def check_coverage(voc,model):
    present = {}
    not_present = {}
    k = 0
    i = 0
    for word in (voc):
        try:
            present[word] = model[word]
            k += voc[word]
        except:
            not_present[word] = voc[word]
            i += voc[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(present) / len(voc)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(not_present.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
not_present = check_coverage(vocab,model_google)

In [ ]:
not_present

In [ ]:
def clean_text_w2v(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [ ]:
X_train_clean_w2v = [clean_text_w2v(line) for line in tqdm.tqdm(X_train)]
vocab = get_vocab(X_train_clean_w2v)
not_present = check_coverage(vocab,model_google)

In [ ]:
X_test_clean_w2v = [clean_text_w2v(line) for line in tqdm.tqdm(X_test)]

In [ ]:
EMBEDDING_DIM =300
word_index = t.word_index
num_words = len(t.word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    try:
        embedding_vector = model_google[word]
        embedding_matrix[i] = embedding_vector
    except:
        None

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

t = Tokenizer()
t.fit_on_texts(X_train_clean_w2v)
X_train_token = t.texts_to_sequences(X_train_clean_w2v)
X_test_token = t.texts_to_sequences(X_test_clean_w2v)
sent_length = 50
X_train_token_pad = pad_sequences(X_train_token,padding='pre',maxlen=sent_length)
X_test_token_pad = pad_sequences(X_test_token,padding='pre',maxlen=sent_length)

In [ ]:
X_train_token_pad, X_val_token_pad, Y_train_token_pad, Y_val_token_pad = train_test_split(X_train_token_pad, Y_train, test_size=0.33, random_state=42)

In [ ]:
Y_train_token_pad

In [ ]:
from keras.utils import np_utils
Y_train_token_pad = np_utils.to_categorical(Y_train_token_pad)
Y_val_token_pad = np_utils.to_categorical(Y_val_token_pad)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten,Dropout,LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.initializers import Constant
max_length = 50
model = Sequential()
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)


model.add(embedding_layer)
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


# fit the model
model.fit(X_train_token_pad, Y_train_token_pad, batch_size=128, epochs=32, validation_data=(X_val_token_pad,Y_val_token_pad), verbose=2)

In [ ]:
Y_pred_token_pre = model.predict_classes(X_test_token_pad)

In [ ]:
submission = pd.DataFrame()
submission['PhraseId'] = df_test.PhraseId
submission['Sentiment'] = Y_pred_token_pre
submission.to_csv('sub.csv', index=False)